Libraries required

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Concatenate, Activation, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical

Functions that create the layers of the proposed architecture - ReLU variant

In [ ]:
def part1():
  model = Sequential([
      # Layer 1
      Conv2D(15, (3, 3), padding='same', input_shape=(224, 224, 3), kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),

      # Layer 2
      Conv2D(30, (3, 3), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2, 2), padding='same'),

      # Layer 3
      Conv2D(45, (3, 3), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),

      # Layer 4
      Conv2D(60, (3, 3), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2, 2), padding='same'),

      # Layer 5
      Conv2D(75, (3, 3), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),

      # Layer 6
      Conv2D(90, (3, 3), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2, 2), padding='same'),
  ])
  return model


def part2():
  model = Sequential([
      # Layer 7
      Conv2D(105, (5, 5), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2,2), padding='same'),
      Dropout(0.1),
      
      # Layer 8
      Conv2D(90, (5, 5), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2,2), padding='same'),
      Dropout(0.1),

      # Layer 9
      Conv2D(75, (5, 5), padding='same', kernel_initializer='he_normal'),
      BatchNormalization(),
      Activation('relu'),
      MaxPooling2D(pool_size=(2,2), padding='same'),
      Dropout(0.1),

      #FC
      Flatten(),
      Dense(1000, kernel_initializer='glorot_normal'),
      Activation('relu'),
      Dense(7, kernel_initializer='glorot_normal'),
      Activation('softmax'),
  ])
  return model

Connection to Google Drive to access the 224x224 variant of the dataset

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Loading the two inputs of the Siamese CNN, the two validation inputs, and the training and validation labels.

In [ ]:
train_a = np.load('/content/gdrive/MyDrive/224X224/x_Train_A.npy')
train_b = np.load('/content/gdrive/MyDrive/224X224/x_Train_B.npy')
labels = np.load('/content/gdrive/MyDrive/224X224/y_Train.npy')
val_a = np.load('/content/gdrive/MyDrive/224X224/x_Test_A.npy')
val_b = np.load('/content/gdrive/MyDrive/224X224/x_Test_B.npy')
val_labels = np.load('/content/gdrive/MyDrive/224X224/y_Test.npy')

Converting training and validation labels into categories.

In [ ]:
labels = to_categorical(labels)
val_labels = to_categorical(val_labels)

print(labels.shape)
print(val_labels.shape)

(1139, 7)
(126, 7)


Definition of the Siamese CNN model

In [ ]:
left_input = Input(shape=(224,224,3))
right_input = Input(shape=(224,224,3))

siamese_model = part1()
left_features = siamese_model(left_input)
right_features = siamese_model(right_input)
print(left_features.shape)

merged_features = tf.concat([left_features, right_features], axis=-1)
print(merged_features.shape)

conv_model = part2()
conv_output = conv_model(merged_features)
print(conv_output.shape)

model = Model(inputs=[left_input, right_input], outputs=conv_output)

(None, 9, 9, 60)
(None, 9, 9, 120)
(None, 7)


Training of the model and definition of checkpoint method to save the best results obtained by the model.

In [ ]:
from keras.callbacks import ModelCheckpoint

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

filepath = "/content/gdrive/My Drive/best_weights_pa_relu.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             monitor='val_loss', 
                             mode='min', 
                             verbose=1)

history = model.fit([train_a, train_b],
           labels,
           epochs=100,
           batch_size=32,
           shuffle=True,
           validation_data=([val_a, val_b], val_labels),
           callbacks=[checkpoint])

Plotting the results of accuracy and loss after the model has finished training

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Observng the highest accuracy and lowest loss values of the model

In [ ]:
best_accuracy = max(history.history['val_accuracy'])

# Find the index of the highest accuracy value
index = history.history['val_accuracy'].index(best_accuracy)

# Get the corresponding loss value
loss_f_best_acc = history.history['val_loss'][index]

print('Best Accuracy:', best_accuracy, 'Loss:', loss_f_best_acc)

best_loss = min(history.history['val_loss'])

# Find the index of the minimum loss value
index = history.history['val_loss'].index(best_loss)

# Get the corresponding loss value
acc_f_min_loss = history.history['val_accuracy'][index]

print('Best Loss:', best_loss, 'Accuracy:', acc_f_min_loss)